# EDS 296 Final Mini Project
#### Authors: Liz Peterson & Brooke Grazda
#### Date: 6-10-25

![amazon-river-basin.jpg](amazon-river-basin.jpg)

### Introduction

The region of interest we are choosing for this assignment is Brazil, home of the Amazon. The Amazon Rainforest is located in the northwestern region of Brazil, with 60% of its coverage in this country. Brazil is home to many culturally and intrinsically rich cultures and species, surrounded by lush and tropical habitats. Brazil is the 5th largest nation in the world, spanning 10,492 mi in South America. It touches every country in South America, except for Ecuador and Chile. This country is also home to a Wonder of the World, Christ the Redeemer located in Rio de Janeiro. Every year, millions flock to this large tropical city in the southeastern end of Brazil. Because of its large area and diverse ecosystems, Brazil's climate spans from equatorial humid climates in the south to semi arid climates in the north east. 


In the [2022 election](https://www.npr.org/2022/10/30/1132561987/brazil-election-lula-da-silva), Luiz Inácio Lula da Silva beat the incumbent president, Bolsonaro, who led a far right, authoritarian presidency. This was a huge win for same sex marriage, abortion, affirmative action, indigenous rights, and environmental protections as Bolsonaro opposed each of these topics. During Bolsonaro's presidency, he rolled back on indigenous rights and facilitated deforestation in the Amazon. Since the election, President Lula de Silva has promoted many fiscal policies and social welfare programs that have led to an increase in national GDP. Much of the environmental damages in the Amazon from Bolsonaro are being reversed under Lula. Despite curbing Bolsonaro's deforestation of the Amazon, Lula has not supported legislation to phase out fossil fuels in the country. 


Understanding the geopolitical context of the region, Brazil becomes a hotspot for extreme climate events to occur, such as floods, droughts, and heatwaves. With 2024 being the hottest year on record, researchers were able to study the 32 year period from 1991 to 2023. From this [data](https://news.mongabay.com/2025/02/brazil-has-seen-a-460-increase-in-climate-related-disasters-since-the-1990s/), it was found that 64,280 climate-related disasters occured in 92% of Brazilian municipalities. Half of the disasters recorded were droughts, while flooding, torrential rains and high water levels composed 27% of the total. Storms composed 19%. More than 219 million people were affected by death, displacement, homelessness and illness. 78 million of these people were affected during the last four years. The average annual economic losses from these climate related disasters since 2020 is $8.14 billion, which is more than double the average annual loss from the previous year. This ecologically sensitive region supports millions of species and communities whose livelihoods depend entirely on the surrounding ecosystem. As climate change accelerates environmental degradation, humans will inevitably bear the mounting costs of diminished ecosystem services.

![<img src="brazil-climate1.jpeg" width="200">](brazil-climate1.jpeg)
![map](map-brazil.jpeg)

### Metrics Definition

We will be doing a correlation and regression analysis of soil moisture content (mrso) and air temperature (tas). 

### Data I/O

In [11]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import intake
import s3fs
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import scipy.stats as stats

In [16]:
pip install intake-esm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.3/32.3 MB 28.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 32.4 MB/s eta 0:00:00a 0:00:0

In [18]:
# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

AttributeError: Unknown open method 'open_esm_datastore'. Do you need to install a new driver from the plugin directory? https://intake.readthedocs.io/en/latest/plugin-directory.html
Registered opener methods: ['open_csv', 'open_jsonfiles', 'open_ndzarr', 'open_numpy', 'open_textfiles', 'open_tiled_cat', 'open_yaml_file_cat', 'open_yaml_files_cat']

### Metrics Computation

### Visualizations

### Conclusions